[1] 모듈 로딩 및 데이터 준비 <hr>

In [2]:
# ==================================================
# [1-1] 모듈 로딩
# ==================================================
# 기본 모듈
import pandas as pd
import numpy as np

# 시각화
import matplotlib.pyplot as plt
import koreanize_matplotlib

# ML 데이터셋 및 학습 관련 모듈
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# ML 모델 성능 지표 관련 모듈
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import root_mean_squared_error, r2_score

# CoW 활성화 : 복사와 공유 자동 설정
pd.options.mode.copy_on_write = True

# 판다스/넘파이에 지수표기법(과학적표기법) 정지 설정
pd.options.display.float_format = '{:.5f}'.format
np.set_printoptions(precision=6, suppress=True)

In [7]:
# ==================================================
# [1-2] 데이터 준비 및 확인
# ==================================================
# 데이터 준비
DATA_FILE = '../Data/world_co2_monthly_1959-01_to_2000-12.csv'
DATA_FILE2 = '../Data/world_co2_monthly_2001-01_to_2013-12.csv'

trainDF = pd.read_csv(DATA_FILE)
testDF  = pd.read_csv(DATA_FILE2)

display(trainDF.head(3))
trainDF.info()

,date,Value
0,1959-01-01,315.58000
1,1959-02-01,316.48000
2,1959-03-01,316.65000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    504 non-null    object 
 1   Value   504 non-null    float64
dtypes: float64(1), object(1)
memory usage: 8.0+ KB
